In [ ]:
%matplotlib widget

from pathlib import Path

import lotr.plotting as pltltr
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
from lotr import A_FISH, FIGURES_LOCATION, LotrExperiment, dataset_folders
from lotr.behavior import get_bouts_props_array, get_fictive_heading
from matplotlib import pyplot as plt
from tqdm import tqdm

COLS = pltltr.COLS

In [ ]:
coll.get_paths()[0].vertices[:] = coll.get_paths()[0].vertices[:] + 1

In [ ]:
coll.get_paths()[0].vertices = coll.get_paths()[0].vertices + 1

In [ ]:
exp = LotrExperiment(A_FISH)
stimulus_log = exp.stimulus_log
stimulus_log.set_index("t")
bouts_df = exp.bouts_df

fict_heading = get_fictive_heading(exp.n_pts, exp.bouts_df)  # / 3
phase = np.unwrap(exp.network_phase)

pink_noise_file = Path("/Users/luigipetrucco/Desktop/pink_noise_norm.png")
pink_noise = mpimg.imread(pink_noise_file)

In [ ]:
exp.bouts_df.columns

In [ ]:
dx = get_bouts_props_array(exp.n_pts, exp.bouts_df, min_bias=0, value="med_vig")
dtheta = get_bouts_props_array(exp.n_pts, exp.bouts_df, min_bias=0, value="bias")

In [ ]:
exp["stimulus"]["log"][2].keys()

In [ ]:
exp["stimulus"]["log"][2]["background_name"]

In [ ]:
def convolve_with_tau(array, tau_fs, n_kernel_pts=1000):

    kernel = np.exp(-np.arange(n_kernel_pts) / tau_fs)
    kernel = kernel / np.sum(kernel)

    return np.convolve(array, kernel)[: len(array)]

In [ ]:
norm_fict_head = -(fict_heading / 1.15)

plt.figure()
plt.plot(norm_fict_head)
plt.plot(convolve_with_tau(norm_fict_head, 30))
plt.plot(phase)

In [ ]:
from lotr.utils import linear_regression

In [ ]:
convolved_head = convolve_with_tau(fict_heading, 20)

t_slice = slice(5180, 6540)
o, c = linear_regression(phase[t_slice], convolved_head[t_slice])

plt.figure(figsize=(5, 2))
plt.plot(phase[t_slice])
plt.plot((convolved_head[t_slice] - o) / c)
plt.title(np.mean(np.abs((convolved_head[t_slice] - o) / c - phase[t_slice])))
plt.tight_layout()

In [ ]:
convolved_head = convolve_with_tau(fict_heading, 20)

t_slice = slice(6100, 6540)
o, c = linear_regression(phase[t_slice], convolved_head[t_slice])

plt.figure(figsize=(5, 2))
plt.plot(phase[t_slice])
plt.plot((convolved_head[t_slice] - o) / c)
plt.title(np.mean(np.abs((convolved_head[t_slice] - o) / c - phase[t_slice])))
plt.tight_layout()

In [ ]:
convolved_head = convolve_with_tau(fict_heading, 30)

t_slice = slice(0, 1300)
o, c = linear_regression(phase[t_slice], convolved_head[t_slice])

plt.figure(figsize=(5, 2))
plt.plot(phase[t_slice])
plt.plot((convolved_head[t_slice] - o) / c)
plt.title(np.mean(np.abs((convolved_head[t_slice] - o) / c - phase[t_slice])))
plt.tight_layout()